In [0]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType
from pyspark.sql.functions import lit, col


# Update the final status of the run
def finalize_run(status_table, job_run_id):
    final_status = dbutils.widgets.get("run_state")
    end_time = dbutils.widgets.get("end_time")
    run_duration = dbutils.widgets.get("run_duration ")
    status_table = status_table.withColumn("End time", 
                                           when(col("Job Run ID") == job_run_id, lit(end_time)).otherwise(col("End time")))
    status_table = status_table.withColumn("Status", 
                                           when(col("Job Run ID") == job_run_id, lit(final_status)).otherwise(col("Status")))
    status_table = status_table.withColumn("Duration", 
                                           when(col("Job Run ID") == job_run_id, 
                                                (col("End time").cast("long") - col("Start time").cast("long"))).otherwise(col("Duration")))
    return status_table


status_table = spark.table("default.status")
job_run_id = dbutils.widgets.get("run_id")

# Finalize the run
status_table = finalize_run(status_table, job_run_id)


#display(status_table)

# Step 5: Store the updated DataFrame as a new table
status_table.write.mode("overwrite").saveAsTable("default.status")